<a href="https://colab.research.google.com/github/davinnnnn/Fine-tuning/blob/main/nsmc_huggingface_koelectra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pytorch + HuggingFace
## KoElectra Model
박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요

In [1]:
# HuggingFace의 transformers 라이브러리를 설치합니다.
!pip install transformers

# NSMC (Naver Sentiment Movie Corpus)의 테스트 데이터셋을 다운로드합니다.
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

# NSMC의 훈련 데이터셋을 다운로드합니다.
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.8 MB/s eta 0:00:00
--2023-10-22 17:39:48--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.1s    

2023-10-22 17:39:48 (47.7 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2023

In [2]:
# `ratings_train.txt` 파일의 첫 10줄을 출력합니다.
!head ratings_train.txt

# `ratings_test.txt` 파일의 첫 10줄을 출력합니다.
!head ratings_test.txt


id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1
id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??	0
7898805	음악이 주가 된, 최고의 음악영화	1
6315043	진정한 쓰레기	0
6097171	마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다	0
8932678	갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다	0


In [3]:
# 필요한 라이브러리들을 임포트합니다.

# 데이터 분석 및 조작을 위한 판다스 라이브러리를 임포트합니다.
import pandas as pd

# 파이토치 라이브러리를 임포트합니다.
import torch
# 파이토치에서 제공하는 함수를 통해 다양한 활성화 함수 등을 사용할 수 있습니다.
from torch.nn import functional as F
# 데이터 로딩을 위한 DataLoader와 사용자 정의 데이터셋을 위한 Dataset 클래스를 임포트합니다.
from torch.utils.data import DataLoader, Dataset

# HuggingFace의 transformers 라이브러리에서 필요한 모델과 토크나이저를 임포트합니다.
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW

# 진행상황을 표시하기 위한 tqdm 라이브러리를 임포트합니다.
from tqdm.notebook import tqdm


In [4]:
# GPU 디바이스를 사용하기 위해 설정합니다. 이 코드를 실행하면 PyTorch는 CUDA를 지원하는 GPU를 사용할 준비를 합니다.
device = torch.device("cuda")

# Dataset 만들어서 불러오기

In [5]:
class NSMCDataset(Dataset):  # PyTorch의 Dataset 클래스를 상속받아 새로운 데이터셋 클래스를 정의합니다.

  def __init__(self, csv_file):
    # 해당 csv 파일에서 데이터를 읽어오고, NaN 값을 포함하는 행을 제거합니다.
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0)
    # 'document' 열에서 중복된 값을 제거합니다.
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    # KoELECTRA의 토크나이저를 불러옵니다.
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    # 데이터셋의 기본 통계 정보를 출력합니다.
    print(self.dataset.describe())

  # 이 데이터셋의 총 길이(데이터 수)를 반환합니다.
  def __len__(self):
    return len(self.dataset)

  # 주어진 인덱스(idx)에 해당하는 데이터를 반환합니다.
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values  # 해당 인덱스의 데이터를 가져옵니다.
    text = row[0]  # 리뷰 텍스트
    y = row[1]  # 라벨 (0 or 1)

    # 텍스트를 토큰화하고, 필요한 형식으로 반환합니다.
    inputs = self.tokenizer(
        text,
        return_tensors='pt',  # PyTorch 텐서로 반환
        truncation=True,  # 길이가 너무 길 경우 잘라냅니다.
        max_length=256,  # 최대 길이
        pad_to_max_length=True,  # 설정된 최대 길이에 맞게 패딩
        add_special_tokens=True  # 특수 토큰(BERT 등의 [CLS], [SEP])을 추가
        )

    input_ids = inputs['input_ids'][0]  # 토큰화된 결과의 ID
    attention_mask = inputs['attention_mask'][0]  # 어텐션 마스크

    return input_ids, attention_mask, y  # 토큰 ID, 어텐션 마스크, 라벨을 반환합니다.


In [6]:
train_dataset = NSMCDataset("ratings_train.txt")  # 훈련 데이터셋을 로드합니다.
test_dataset = NSMCDataset("ratings_test.txt")    # 테스트 데이터셋을 로드합니다.


                 id          label
count  1.461820e+05  146182.000000
mean   6.779186e+06       0.498283
std    2.919223e+06       0.499999
min    3.300000e+01       0.000000
25%    4.814832e+06       0.000000
50%    7.581160e+06       0.000000
75%    9.274760e+06       1.000000
max    1.027815e+07       1.000000
                 id         label
count  4.915700e+04  49157.000000
mean   6.752945e+06      0.502695
std    2.937158e+06      0.499998
min    6.010000e+02      0.000000
25%    4.777143e+06      0.000000
50%    7.565415e+06      1.000000
75%    9.260204e+06      1.000000
max    1.027809e+07      1.000000


# Create Model

In [7]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
# KoELECTRA 모델을 불러온 후, GPU로 이동시킵니다.

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))
# 훈련 데이터셋의 첫 번째 샘플에 대해 모델을 실행시켜 보는 코드입니다.


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# model.load_state_dict(torch.load("model.pt"))

In [9]:
# 모델의 구조 및 레이어를 출력합니다.
model


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

# Learn

In [10]:
# 학습 시 사용할 에폭 수와 배치 크기를 설정합니다.
epochs = 1
batch_size = 16


In [11]:
# 모델을 최적화하기 위해 AdamW 옵티마이저를 사용하고 데이터 로더를 설정합니다.
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
losses = []  # 각 에폭의 평균 손실을 저장
accuracies = []  # 각 에폭의 정확도를 저장

# 주어진 에폭 수만큼 학습을 반복합니다.
for i in range(epochs):
    total_loss = 0.0  # 현재 에폭의 총 손실
    correct = 0  # 현재 에폭의 올바르게 예측된 샘플 수
    total = 0  # 현재 에폭의 총 샘플 수
    batches = 0  # 처리된 배치 수

    # 모델을 학습 모드로 설정합니다.
    model.train()

    # 학습 데이터셋의 각 배치에 대해 학습을 진행합니다.
    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()  # 이전 배치의 기울기를 초기화합니다.
        y_batch = y_batch.to(device)  # 라벨을 GPU로 옮깁니다.

        # 모델을 통해 예측을 실행합니다.
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]

        # 손실을 계산합니다.
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()  # 역전파를 통해 기울기를 계산합니다.
        optimizer.step()  # 모델의 가중치를 업데이트합니다.

        total_loss += loss.item()  # 총 손실을 누적합니다.

        _, predicted = torch.max(y_pred, 1)  # 예측된 클래스를 가져옵니다.
        correct += (predicted == y_batch).sum()  # 올바르게 예측된 샘플 수를 누적합니다.
        total += len(y_batch)  # 총 샘플 수를 누적합니다.

        batches += 1  # 처리된 배치 수를 누적합니다.
        if batches % 100 == 0:  # 100 배치마다 로그를 출력합니다.
            print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

    # 에폭의 평균 손실과 정확도를 저장합니다.
    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)


  0%|          | 0/9137 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 69.42347764968872 Accuracy: tensor(0.4912, device='cuda:0')
Batch Loss: 138.52093589305878 Accuracy: tensor(0.5094, device='cuda:0')
Batch Loss: 207.20451366901398 Accuracy: tensor(0.5206, device='cuda:0')
Batch Loss: 274.8531247973442 Accuracy: tensor(0.5366, device='cuda:0')
Batch Loss: 341.0803721547127 Accuracy: tensor(0.5535, device='cuda:0')
Batch Loss: 406.21731412410736 Accuracy: tensor(0.5633, device='cuda:0')
Batch Loss: 469.30702099204063 Accuracy: tensor(0.5743, device='cuda:0')
Batch Loss: 530.970300912857 Accuracy: tensor(0.5866, device='cuda:0')
Batch Loss: 592.6703405380249 Accuracy: tensor(0.5941, device='cuda:0')
Batch Loss: 653.2608780562878 Accuracy: tensor(0.6011, device='cuda:0')
Batch Loss: 713.0342999100685 Accuracy: tensor(0.6086, device='cuda:0')
Batch Loss: 772.2145787775517 Accuracy: tensor(0.6151, device='cuda:0')
Batch Loss: 828.6489257216454 Accuracy: tensor(0.6222, device='cuda:0')
Batch Loss: 885.4363981485367 Accuracy: tensor(0.6278, device

In [1]:
# 훈련 과정에서 각 에포크별로 기록된 손실(loss)과 정확도(accuracy)를 출력
losses, accuracies


NameError: ignored

테스트 데이터셋 정확도 확인하기

In [ ]:
# 모델을 평가 모드로 변경
model.eval()

# 정확도를 계산하기 위한 변수 초기화
test_correct = 0
test_total = 0

# 테스트 데이터셋에 대한 반복
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  # 현재 배치의 레이블을 GPU로 이동
  y_batch = y_batch.to(device)

  # 모델을 통해 예측값 계산
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]

  # 가장 높은 확률을 가진 클래스 예측
  _, predicted = torch.max(y_pred, 1)

  # 예측값과 실제값이 같은 경우의 수를 합산
  test_correct += (predicted == y_batch).sum()

  # 전체 레이블 수를 합산
  test_total += len(y_batch)

# 테스트 정확도 출력
print("Accuracy:", test_correct.float() / test_total)


In [ ]:
# 모델의 파라미터(가중치 및 편향)를 "model.pt" 파일에 저장
torch.save(model.state_dict(), "model.pt")
